In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import pause

In [ ]:
# Capture a screenshot of the device's screen and pull the captured screenshot from the device to the local machine
os.system('adb shell screencap -p /sdcard/screencap.png')
os.system('adb pull /sdcard/screencap.png')

In [ ]:
# Read the image using OpenCV
img = cv2.imread('screencap.png')
screen_height, screen_width, _ = img.shape

# Set the size of the displayed plot using Matplotlib
plt.rcParams["figure.figsize"] = [15,8]
plt.imshow(img)

# Convert the color space of the image from BGR (OpenCV's default) to RGB for proper display with Matplotlib
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

In [ ]:
# Get the dimensions of the image and normalise the height and width
yf, xf, zf = np.shape(img)
yf = yf/2280
xf = xf/1080

In [ ]:
# Crop the image based on specified y-axis (vertical) coordinates and retain all columns
img_crop = img[int(np.floor(1550*yf)):int(np.floor(1650*yf)), :]
plt.imshow(img_crop)
y_crop, x_crop, z_crop = np.shape(img_crop)

In [ ]:
# Set all the RGB values to ZERO giving black color   """ 
my_img = np.zeros_like(img_crop)
plt.imshow(my_img)

In [ ]:
rcord=set()

# Iterate over each pixel in the cropped image
for j in range(y_crop):
    for k in range(x_crop):
        # Check if the pixel color matches either red (255, 0, 0) or a similar shade
        if(((img_crop[j,k,0]==255) and (img_crop[j,k,1]==0) and (img_crop[j,k,2]==0)) or ((img_crop[j,k,0]==247) and (img_crop[j,k,1]==27) and (img_crop[j,k,2]==27))):
            # Set the corresponding pixel in the new image to green (0, 255, 0)
            my_img[j,k,0] = 0
            my_img[j,k,1] = 255
            my_img[j,k,2] = 0
            rcord.add(k)  

In [ ]:
plt.imshow(my_img)

In [ ]:
# Sort the recorded x-coordinates in ascending order
rcord = sorted(rcord)
print(rcord)

In [ ]:
def srcTarget(rcord):
    prev=rcord[0]
    for i in rcord:
        # Check if the absolute difference between the current and previous x-coordinates is greater than 10
        if abs(i-prev) > 10:
            # Return the current and previous x-coordinates as source and target points respectively
            # Add 5 to the previous x-coordinate to roughly compensate for the extra distance where the stick rises
            return i, prev + 5    
        prev=i

In [ ]:
target, src = srcTarget(rcord)
print(src,target)

In [ ]:
dist = abs(src-target)
rate = 1.045* (screen_width/1080)
print(rate)
time = str(int(dist/rate))

print(dist,time)

In [ ]:
# Fix some x and y coordinates to swipe on the screen, with time already calculated before
x = str(int(np.ceil(530*yf)))
y = str(int(np.ceil(830*xf)))

os.system('adb shell input touchscreen swipe ' + x + ' ' + x + ' ' + y + ' ' + y + ' ' + time)
pause.seconds(3)

In [ ]:
rate = 0.51

while True:

    os.system('adb shell screencap -p /sdcard/screencap.png')
    os.system('adb pull /sdcard/screencap.png')

    img = cv2.imread('screencap.png')
    screen_height, screen_width, _ = img.shape

    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    yf, xf, zf = np.shape(img)
    yf = yf/2280
    xf = xf/1080
    
    img_crop = img[int(np.floor(1550*yf)):int(np.floor(1700*yf)), :]
    plt.imshow(img_crop)
    y_crop, x_crop, z_crop = np.shape(img_crop)

    my_img = np.zeros_like(img_crop)

    rcord=set()
    
    for j in range(y_crop):
        for k in range(x_crop):
            if(((img_crop[j,k,0]==255) and (img_crop[j,k,1]==0) and (img_crop[j,k,2]==0)) or ((img_crop[j,k,0]==247) and (img_crop[j,k,1]==27) and (img_crop[j,k,2]==27))):
                my_img[j,k,0] = 0
                my_img[j,k,1] = 255
                my_img[j,k,2] = 0
                rcord.add(k)  

    rcord = sorted(rcord)
    target, src = srcTarget(rcord)
    print(src,target)
    
    dist = abs(src-target)
    rate = 1.0475 * (screen_width/1080)
    time = str(int(dist/rate))
    
    print(dist,time)

    x = str(int(np.ceil(530*yf)))
    y = str(int(np.ceil(830*xf)))

    os.system('adb shell input touchscreen swipe ' + x + ' ' + x + ' ' + y + ' ' + y + ' ' + time)
    pause.seconds(3)